<a href="https://colab.research.google.com/github/tom5610/my-fsdl-text-recognizer-2022/blob/main/lab01/notebooks/c_lab01_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 01: Deep Neural Networks in PyTorch

Objective:
* how to write a basic neural network from scatch in PyTorch
* How the submodules of torch, like `torch.nn` and `torch.utils.data`, make writing performant neural network training and inference code easier

## Setup
Below cell will run full environment setup in colab.

In [1]:
lab_idx = 1

if 'bootstrap' not in locals() or bootstrap.run:
    pythonpath, = !echo $PYTHONPATH
    if '.' not in pythonpath.split(':'):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow 'hot-reloading' of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


## Getting data and making Tensors

before we can build a model, we need data. 

Use MNIST dataset of handwritten digits.

The data used to train state-of-the-art models these days is generally too large to be stored on the disk of any single machine (to say nothing of the RAM!), so fetching data over a network is a common first step in model training.

In [2]:
from pathlib import Path 
import requests

def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [3]:
import gzip
import pickle 

def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)
    

In [4]:
import torch 

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [5]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [6]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [7]:
x_train.ndim, y_train.ndim

(2, 1)

In [8]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [9]:
import random 

import wandb 

import text_recognizer.metadata.mnist as metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])
wandb.Image(example.reshape(*metadata.DIMS)).image

tensor(1)


## Build a DNN using only `torch.Tensor` methods and Python

### Defining the model

In [10]:
import math

import torch
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [11]:
weights.ndim, weights.shape

(2, torch.Size([784, 10]))

In [12]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [13]:
n = 3
a = torch.rand(n)
b = torch.rand(n)
print(torch.matmul(a, b))
print(a @ b)
a, b

tensor(0.5357)
tensor(0.5357)


(tensor([0.8534, 0.0395, 0.1505]), tensor([0.5625, 0.8993, 0.1341]))

In [14]:
0.5367 * 0.0799 + 0.361 * 0.3948 + 0.3556 * 0.6106

0.40253448999999997

In [15]:
a = torch.rand((3, 2))
b = torch.rand((2, 2))
result = a.mm(b)

In [16]:
a, b, result

(tensor([[0.1425, 0.8541],
         [0.2732, 0.6872],
         [0.6426, 0.2616]]), tensor([[0.6202, 0.0251],
         [0.3230, 0.1609]]), tensor([[0.3642, 0.1410],
         [0.3914, 0.1174],
         [0.4830, 0.0582]]))

In [17]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis = 1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [18]:
bs = 64 # batch size

xb = x_train[0: bs] # a batch of inputs
outs = model(xb)

print(outs[0], outs.shape)


tensor([-2.3745, -3.1393, -2.3581, -2.2854, -2.1499, -3.1087, -2.3167, -2.1020,
        -1.6897, -2.2952], grad_fn=<SelectBackward0>) torch.Size([64, 10])


## Defining the loss and metrics

In [19]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    print('accuracy comparison - ', (preds == yb).float())
    return (preds == yb).float().mean()

In [20]:
yb = y_train[0: bs]

acc = accuracy(outs, yb)

print(acc)

accuracy comparison -  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor(0.0938)


In [21]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [22]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.4199, grad_fn=<NegBackward0>) tensor(2.3026)


In [23]:
torch.log(torch.tensor(1 / 10))

tensor(-2.3026)

In [24]:
loss = loss_func(outs, yb)

loss.backward()

In [25]:
weights.grad, bias.grad

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([ 0.0012, -0.0519,  0.0194, -0.0198, -0.0352,  0.0182, -0.0047,  0.0488,
          0.0525, -0.0286]))

## Defining and running the fitting loop

We now have all the ingredients we need to fit a neural network to data:

* data (x_train, y_train)
* a network architecture with parameters (model, weights, and bias)
* a loss_func to optimize (cross_entropy) that supports .backward computation of gradients

In [26]:
lr = 0.5
epochs = 2

for epoch in range(epochs): # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad(): # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # achtung: PyTorch doesn't assume you're done with gradients
            #.         until you say so -- by explicitly "deleting" them. i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()


In [27]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(1.0131, grad_fn=<NegBackward0>) tensor(0.8281)


In [28]:
idx = random.randint(0, len(x_train))
example = x_train[idx: idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(1)


## Refactoring with core `torch.nn` components


### Using `torch.nn.functional` for stateless computation

In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [29]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(1.0131, grad_fn=<NegBackward0>) tensor(0.8281)


### Using torch.nn.Module to define functions whose state is given by `torch.nn

In [34]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() # this is mandatory as nn.Module does import setup
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [37]:
def forward(self, xb:torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()
print(model(xb)[:4])
loss = loss_func(model(xb), yb)
loss.backward()
print(model.weights.grad[::17, ::2])

tensor([[-2.7356e-01, -6.1284e-01, -4.4267e-01,  1.0994e-01,  1.6760e-01,
          1.8876e-01,  1.8824e-01,  7.0798e-02, -2.7284e-01,  6.4952e-01],
        [-2.7472e-01,  1.8758e-01, -7.9060e-01,  6.3794e-01, -3.8115e-01,
          5.8444e-02,  2.4837e-01, -1.2672e-01, -1.3872e-01,  2.7652e-01],
        [-3.7612e-01, -2.5516e-01, -5.4322e-01, -1.3624e-01,  3.7994e-01,
         -1.1055e-01,  6.5418e-01,  4.4629e-01, -4.0257e-01,  9.3307e-02],
        [-5.4124e-04, -3.2731e-01,  3.7658e-02,  2.3763e-02, -1.6391e-03,
          1.6479e-01, -5.0025e-01, -8.9057e-03, -5.8572e-01, -1.0472e-01]],
       grad_fn=<SliceBackward0>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.0

In [38]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0411,  0.0403, -0.0381,  ...,  0.0070, -0.0098,  0.0361],
        [ 0.0602, -0.0403,  0.0220,  ..., -0.0329,  0.0394, -0.0038],
        [ 0.0049,  0.0048, -0.0598,  ..., -0.0116, -0.0454,  0.0693],
        ...,
        [-0.0292, -0.0112,  0.0322,  ..., -0.0356,  0.0140,  0.0085],
        [ 0.0468,  0.0525, -0.0084,  ...,  0.0426,  0.0116,  0.0396],
        [ 0.0223, -0.0123,  0.0297,  ...,  0.0247,  0.0149, -0.0177]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [39]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs 
            end_idx = start_idx + bs
            yb = x_train[start_idx: end_idx]
            yb = y_train[start_idx: end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [40]:
print(accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 0., 1., 1., 0., 1.])
tensor(0.5625)


In [41]:
import textwrap 

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")


torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [42]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() 
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

In [43]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))

tensor(-0.0012, grad_fn=<NegBackward0>)


In [44]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [45]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0321, -0.0333,  0.0050,  ...,  0.0101,  0.0331, -0.0049],
        [-0.0077, -0.0034,  0.0054,  ..., -0.0198,  0.0309, -0.0180],
        [ 0.0281, -0.0311,  0.0048,  ..., -0.0281,  0.0283,  0.0092],
        ...,
        [-0.0223,  0.0130, -0.0346,  ..., -0.0071,  0.0277,  0.0103],
        [-0.0292,  0.0346, -0.0226,  ..., -0.0108,  0.0044,  0.0176],
        [-0.0288, -0.0064, -0.0314,  ...,  0.0090, -0.0227, -0.0341]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0037, -0.0020, -0.0081,  0.0126,  0.0151, -0.0207,  0.0318, -0.0309,
         0.0261, -0.0084], requires_grad=True)


### Applying gradients with `torch.optim.Optimizer`

In [46]:
from torch import optim 

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [48]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs 
        end_idx = start_idx + bs 
        xb = x_train[start_idx: end_idx]
        yb = y_train[start_idx: end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training: ", loss_func(model(xb), yb), sep="\n\t")

after training: 
	tensor(-0.1129, grad_fn=<NegBackward0>)


In [49]:
from text_recognizer.data.util import BaseDataset 

train_ds = BaseDataset(x_train, y_train)

In [50]:
BaseDataset??

In [52]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.0718, grad_fn=<NegBackward0>)


### Batching up data with `torch.utils.data.DataLoader`

In [53]:
from torch.utils.data import DataLoader

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size = bs)

In [55]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit 

In [56]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(-39.3669, grad_fn=<NegBackward0>)


### Swapping in another model

In [57]:
from text_recognizer.models.mlp import MLP 

MLP.fit = fit

In [58]:
MLP.forward??

In [59]:
MLP.__init__??

In [64]:
MLP?

In [60]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [61]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [66]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0284, -0.0277,  0.0339,  ..., -0.0036,  0.0009,  0.0147],
        [ 0.0134,  0.0039,  0.0168,  ..., -0.0168,  0.0210,  0.0280],
        [-0.0311,  0.0021, -0.0269,  ..., -0.0355,  0.0220, -0.0293],
        ...,
        [-0.0294,  0.0124, -0.0340,  ...,  0.0169,  0.0009,  0.0283],
        [-0.0285, -0.0297, -0.0071,  ..., -0.0136,  0.0251,  0.0095],
        [ 0.0084, -0.0273,  0.0097,  ...,  0.0005,  0.0007,  0.0239]],
       requires_grad=True)

In [67]:
%%time

print("before training: ", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training: ", loss_func(model(xb), yb))

before training:  tensor(-0.0365, grad_fn=<NegBackward0>)
after training:  tensor(-6013567., grad_fn=<NegBackward0>)
CPU times: user 24 s, sys: 581 ms, total: 24.6 s
Wall time: 24.6 s
